In [1]:
import tensorflow as tf
import keras
from keras import backend as K
from tensorflow.python.client import device_lib
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras import layers
import pandas as pd
import re
import requests
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU, LSTM
from keras.layers import Embedding
import re
import math
import random
from collections import defaultdict
from keras.models import model_from_json
import h5py
import numpy as np
import sentencepiece as spm
import collections
from keras.layers import Dropout
from keras import backend as K
from keras.callbacks import EarlyStopping
import os
import matplotlib.pyplot as plt
import codecs
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

print(tf.__version__)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))


2.4.1
Found GPU at: /device:GPU:0


In [ ]:
# Text generation is at the bottom of the notebook
#
#

In [2]:
# Load text and do some final cleaning

# Change text to desired text
text = '1000000_0_0'

file = 'CLEAN_' + text + '.txt'
path = 'data/' + text + '/'

with open(path + file, encoding="utf8") as handle:
    sentences =  [l.strip()[0:-1] for l in handle.readlines()]
    
sentences1 = []
for sent in sentences:
    sent = re.sub(r"\s+", " ", sent)
    sentences1.append(sent.strip())

x = []

for i in sentences1:
    if len(i.split()) > 2 and len(i.split()) < 50:
        x.append(i)

In [3]:
# load sentencepiece model
sp = spm.SentencePieceProcessor(model_file=path + 'encoding.model')
sp.SetEncodeExtraOptions('bos:eos')

# vocab size
v = 6000

In [4]:
# Encode sentences, and split them the way I describe in my thesis

X = []
X1 = []
y = []
for sentence in x:
    count = 1
    temp = sp.encode(sentence)
    for subword in temp:
        X.append(temp[0:count])
        if count < len(temp):
            y.append(temp[count])
        count += 1
        
for A in X:
    if 2 not in A:
        X1.append(A)

X = X1
y = to_categorical(y, num_classes=v)
print(len(X))

1557644


In [5]:
# Remove datapoints with more than 40 subwords
count = 0
max1 = 40
X1 = []
y1 = []

for XD in range(len(X)):
    if len(X[XD]) < max1:
        count += len(X[XD])
        X1.append(X[XD])
        y1.append(y[XD])
        
X = X1
y = y1
print(count)
print(len(X))
print(len(y))

21647507
1458245
1458245


In [6]:
# Pad sequences

X1 = []
for sent in X:
    X1.append(pad_sequences([sent], maxlen=max1, padding='pre'))
    
X2 = []
for sent in X1:
    X2.append(sent[0])

X = X2
X2 = []
X1 = []
np.shape(X)

(1458245, 40)

In [7]:
# Split the train and test data with a ratio of 1 to 4
X = np.asarray(X)
y = np.asarray(y)

X_train = X[:int(len(X)*0.8)]
X_test = X[int(len(X)*0.8):]
X = []

y_train = y[:int(len(y)*0.8)]
y_test = y[int(len(y)*0.8):]
y = []
print("Done")

Done


In [8]:
# Early stopping
np.set_printoptions(precision=None, suppress=True)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)

In [ ]:
# My own grid search implementation
print(text)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1, restore_best_weights=True)

batches = [256]
dropout = [0.1, 0.2, 0.4]
lr = [0.001, 0.002]
layer_size = [256]

# Meshgrid creates arrays of every possible combinations
grid = np.array(np.meshgrid(batches,dropout,lr, layer_size)).T.reshape(-1, 4)

# for every combination in grid...
for element in grid:
    batch = int(element[0])
    layer = int(element[3])

    optimizer = keras.optimizers.Adam(lr=element[2], beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    
    model = Sequential()
    model.add(Embedding(v, 200, input_length = max1))
    
    # if CuDNN doesnt work, try: model.add(LSTM(layer))
    model.add(CuDNNLSTM(layer, return_sequences=True))
    model.add(Dropout(element[1]))
    model.add(CuDNNLSTM(layer))
    model.add(Dense(v, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    print("Now fitting parameters: " + str(element))

    history = model.fit(X_train, y_train, epochs=100, verbose=1, batch_size=batch, callbacks=[early_stopping], validation_data=(X_test, y_test))



In [48]:
# Load a model from the data to generate text
# Only have to change text name to desired text
text = '1000000_2_5'

sp = spm.SentencePieceProcessor(model_file= 'data/'+ text+ '/encoding.model')
sp.SetEncodeExtraOptions('bos:eos')
json_file = open('data/' +text + '/model/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('data/' +text + '/model/modelcheckP.h5')
print("Loaded model from disk")

Loaded model from disk


In [44]:
# Generate words deterministically (no randomness), can often get stuck in a loop, especially with the non-zipfian trained models
# Appends a '.' when the model predicts an EOS.

max1 = 40

def generate_words_deter(model, text, num):
    for _ in range(num):
        padded_words = pad_sequences([text], maxlen=max1, padding='pre')
        a = np.argsort(model.predict_proba(padded_words)[0])[-2:]
        if a[0] == text[-1] or a[0] == 0:
            b = int(a[1])
        else:
            b = int(a[0])
            
            # Keeps the high typicality models from predicting too many EOS
            if b == 2 and b not in text[-4:]:
                text.append(2)
                text.append(3)
                text.append(1)
                continue
        text.append(b)
    return sp.decode(text)

In [45]:
# Generate words with a random modifier, that probabillistically picks a word from the top X words.
# Appends a '.' when the model predicts an EOS.

max1 = 40
def generate_words_prob(model, text, num):
    for _ in range(num):
        
        padded_words = pad_sequences([text], maxlen=max1, padding='pre')
        abc = model.predict_proba(padded_words)[0]
        a = list(reversed(np.argsort(abc)[-4:]))
        
        weights = sorted(abc, reverse=True)[0:4]
        
        if 0 in a:
            XD = a.index(0)
            a.pop(XD)
            weights.pop(XD)
            
        b = int(random.choices(a,weights=weights)[0])
        
        # Keeps the high typicality models from predicting too many EOS
        if b == 2 and b not in text[-4:]:
            text.append(2)
            text.append(3)
            text.append(1)
            continue
            
        text.append(b)

    return sp.decode(text)

In [32]:
# You can give the model some context to generate from, although it'll go off on a tangent of its own quite quickly

sentence = "den norske byen" # the Norwegian city
sentence = sp.encode(sentence)[:-1] 

generate_words_deter(model, sentence, 4)

'den norske byens hovedsy'

In [49]:
# Or let the model generate from nothing, and just give it a BOS token

generate_words_deter(model, [1], 400)

'han ble valgt inn på stortinget i 2006 og spilte en ny tittelstyring i den amerikansk prøyssisk og krigen og var med i det tyske landslaget i usa og ble tildelt de olympisk rekord. dermed i norge og var en av norges mest berømte utøverne i den internasjonale finalen i 1999 og ble slått til fange. den tyske sosialdemokratiet i norge og ble valgt inn på stortingets landsstyr. den lokale arkitekt. ernst bowie is the newss the newss the newss som var med i det amerikansk ungarsk. det amerikansk ungarsk. han ble valgt som en av norges mest berømte bidrag. de to første i norge i den andre. det var i tillegg ikke en selvstendig skikkelse for de avdøde og de fleste av de viktigste protestantisk og i norge og de fleste andre. den hellige stolte i tillegg. det var en viktig delstat. et stort publikumssted. et annet slag. et annet slag. et annet slag. et annet marked i esbrungspark. den andre seremonisstyrken. essens stadt og en annen forening som var i stand fra en annen rekke andre. den andre 

In [50]:
# Same thing for probablistic methods

sentence = "den norske byen" # the Norwegian city
sentence = sp.encode(sentence)[:-1] 

generate_words_prob(model, sentence, 4)

'den norske byen er en av flere'

In [51]:
generate_words_prob(model, [1], 500)

'han ble kreert til kardinal i 1857 og i 1674. hans hustru ingebrigssy og hans bror av england. den hellige navn. i den første verdenskrig. den andre verdenskrig brøt han i den tyske hæren. den hellige stol. en av de mest populære faktorene i paris. en del av den tyske kirke. den hellige stol. en del av en annen del av en del av den tyske okkupasjonsmakten i den tyske hæren. det ble bygget i 1730. det som var en del av den tyske hæren. den andre verdenskrigen ble bygget. det var en viktig del av de allierte. dermed ble det også en del av det tyske riket. irak og det ble også i. en rekke andre klubber i norge. en del av en rekke andre klubber. en rekke kjente personer som ble utgitt på single. den sørlige delen av fjorden. stein. en av de største idealene.eppes. irak. irakene. en del av en del av en del av den tyske okkupasjonsmakten. den andre verdenskrig brøt ut i et eget departement for den tyske okkupasjonsmakten. den nye okkupasjonsmakten. i tillegg til å organisere et nytt byggomr

In [25]:
# Measure perplexity of model, you'll need to load all the correct data for this to work
# It looks a little complicated because everything had to be done in batches, or else it would crash

def perplexity(Xp, yp, model):
    chance = 0
    current_index = 0
    batch_size = 500
    rang = int(len(Xp) / batch_size) - 1
    length = batch_size * rang
    for _ in range(rang):
        distributionList = model.predict(Xp[current_index:current_index + batch_size], batch_size)
    
        tempcount = current_index
        for distribution in distributionList:
            chance += math.log2(distribution[np.nonzero(yp[tempcount])[0][0]])
            tempcount += 1
            
        current_index = current_index + batch_size 
    return 2**((-1/length)* chance)

perplexity(X_test, y_test, model)

198.85458981015